In [1]:
import os
import sys
import json
import time
import requests

from dotenv import load_dotenv

def setup_env():
    
    env_path = os.path.join(os.getcwd(), '../.env')

    if os.path.exists(env_path):
        load_dotenv(dotenv_path=env_path)
        
        print(f"Loaded environment variables from: \033[94m{env_path}\033[0m")
    else:
            print("\033[91mError: .env file not found. Please create one with your OPENAI_API_KEY.\033[0m")
            sys.exit(1)

setup_env()

Loaded environment variables from: /home/ras/0.agent_ai_ws/src/learn_rag_and_agent/learn_rag_and_agent/../.env


In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

en_template = """
당신은 한영 번역 전문가입니다. 주어진 한국어 문장을 자연스러운 영어로 번역해 주세요.
단순한 직역이 아닌, 영어 원어민이 실제로 사용하는 자연스러운 표현과 뉘앙스를 살려서 번역해 주세요.

#한국어:
{question}

#FORMAT:
- 영어 번역:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
en_prompt = PromptTemplate.from_template(en_template)

# ChatOpenAI 챗모델을 초기화합니다.
en_model = ChatOpenAI(model_name="gpt-4.1-nano")

# 문자열 출력 파서를 초기화합니다.
en_output_parser = StrOutputParser()

In [3]:
# 체인을 구성합니다.
en_chain = en_prompt | en_model | en_output_parser

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

jp_template = """
당신은 한일 번역 전문가입니다. 주어진 한국어 문장을 자연스러운 일본어로 번역해 주세요.
단순한 직역이 아닌, 일본어 원어민이 실제로 사용하는 자연스러운 표현과 뉘앙스를 살려서 번역해 주세요.

#한국어:
{question}

#FORMAT:
- 일본어 번역:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
jp_prompt = PromptTemplate.from_template(jp_template)

# ChatOpenAI 챗모델을 초기화합니다.
jp_model = ChatOpenAI(model_name="gpt-4.1-nano")

# 문자열 출력 파서를 초기화합니다.
jp_output_parser = StrOutputParser()

In [5]:
jp_chain = jp_prompt | jp_model | jp_output_parser

In [6]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print("=== English Translation ===")
print(en_chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

print("=== Japanese Translation ===")
print(jp_chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

=== English Translation ===
- 영어 번역: I want to go to a restaurant and order some food.
=== Japanese Translation ===
- 日本語訳：私はレストランに行って、料理を注文したいです。


In [25]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print("=== English Translation ===")
print(en_chain.invoke({"question": "여기 피자 얼마에요?"}))

print("=== Japanese Translation ===")
print(jp_chain.invoke({"question": "여기 피자 얼마에요?"}))

=== English Translation ===
- 영어 번역: How much is this pizza?
=== Japanese Translation ===
- 日本語訳：
ここはピザいくらですか？


---
- 위와 같이 여러개의 Chain을 만들어서 서로 연결해서 사용할 수 있다
---

In [8]:
from langchain_core.runnables import RunnableParallel

combined_trans = RunnableParallel(en_trans=en_chain, jp_trans=jp_chain)

In [9]:
# 병렬 실행 체인을 실행합니다.
combined_trans.invoke({"question": "여기 피자 얼마에요?"})

{'en_trans': '- 영어 번역: How much is the pizza here?',
 'jp_trans': '- 일본어 번역:\nすみません、こちらのピザはいくらですか？'}

'November 27'

'오늘의 날짜는 November 27 입니다. 오늘이 생일인 유명인 3명을 나열해 주세요. 생년월일을 표기해주세요.'